# Problem 236: Luxury Hampers

Suppliers 'A' and 'B' provided the following numbers of products for the luxury hamper market:

<center><table class="p236"><tr><th>Product</th><th class="center">'A'</th><th class="center">'B'</th></tr><tr><td>Beluga Caviar</td><td>5248</td><td>640</td></tr><tr><td>Christmas Cake</td><td>1312</td><td>1888</td></tr><tr><td>Gammon Joint</td><td>2624</td><td>3776</td></tr><tr><td>Vintage Port</td><td>5760</td><td>3776</td></tr><tr><td>Champagne Truffles</td><td>3936</td><td>5664</td></tr></table></center>

Although the suppliers try very hard to ship their goods in perfect condition, there is inevitably some spoilage - <i>i.e.</i> products gone bad.

The suppliers compare their performance using two types of statistic:</p><ul><li>The five <i>per-product spoilage rates</i> for each supplier are equal to the number of products gone bad divided by the number of products supplied, for each of the five products in turn.</li>
  <li>The <i>overall spoilage rate</i> for each supplier is equal to the total number of products gone bad divided by the total number of products provided by that supplier.</li></ul><p>To their surprise, the suppliers found that each of the five per-product spoilage rates was worse (higher) for 'B' than for 'A' by the same factor (ratio of spoilage rates), <var>m</var>&gt;1; and yet, paradoxically, the overall spoilage rate was worse for 'A' than for 'B', also by a factor of <var>m</var>.

There are thirty-five <var>m</var>&gt;1 for which this surprising result could have occurred, the smallest of which is 1476/1475.

What's the largest possible value of <var>m</var>?\
Give your answer as a fraction reduced to its lowest terms, in the form <var>u</var>/<var>v</var>.

In [1]:
from collections import defaultdict
from fractions import Fraction
from math import ceil

N_A = [5248, 1312, 2624, 5760, 3936]
N_B = [640, 1888, 3776, 3776, 5664]

five_per_product_ms = defaultdict(lambda: [[], [], [], [], []])

for i, (_N_A, _N_B) in enumerate(zip(N_A, N_B)):
    for n_A in range(1, _N_A + 1):
        b_min = Fraction(n_A * _N_B, _N_A)
        b_min = b_min.numerator + 1 if b_min.is_integer() else ceil(b_min)
        for n_B in range(b_min, N_B[i] + 1):
            m = Fraction(n_B * _N_A, n_A * _N_B)
            five_per_product_ms[m][i].append(n_A)

# filter out solutions where some products have no spoilage
five_per_product_ms = dict(
    filter(lambda pair: min(len(_) for _ in pair[1]) > 0, five_per_product_ms.items())
)

In [2]:
from typing import List


def get_overall_spoilage_rate(five_per_product_m: Fraction, n_As: List[int]):
    N_Bs = sum(N_B)
    N_As = sum(N_A)
    m_num = five_per_product_m.numerator
    m_denom = five_per_product_m.denominator
    n_Bs = [
        m_num * n_A * _N_B // (_N_A * m_denom)
        for n_A, _N_A, _N_B in zip(n_As, N_A, N_B)
    ]
    overall_spoilage_rate = Fraction(sum(n_As) * N_Bs, sum(n_Bs) * N_As)
    return overall_spoilage_rate

In [3]:
# filter out solutions max/min overall spilage rate are outside the target rate


def filter_rate(pairs):
    five_per_product_rate, n_As = pairs
    max_ratio = get_overall_spoilage_rate(
        m, [max(n_As[0]), min(n_As[1]), min(n_As[2]), min(n_As[3]), min(n_As[4])]
    )
    min_ratio = get_overall_spoilage_rate(
        m, [min(n_As[0]), max(n_As[1]), max(n_As[2]), max(n_As[3]), max(n_As[4])]
    )
    if min_ratio <= five_per_product_rate <= max_ratio:
        return True
    else:
        return False


five_per_product_ms = dict(filter(filter_rate, five_per_product_ms.items()))

In [4]:
print([Fraction(n_A, n_B) for n_A, n_B in zip(N_A, N_B)])

[Fraction(41, 5), Fraction(41, 59), Fraction(41, 59), Fraction(90, 59), Fraction(41, 59)]


In [5]:
from itertools import product


def search_overall_spoilage_rate_solutions(
    five_per_product_rate: Fraction, n_As: List[int]
):
    N_Bs = sum(N_B)
    N_As = sum(N_A)
    m_num = five_per_product_rate.numerator
    m_denom = five_per_product_rate.denominator
    # N_A/N_B for index 1, 2, 4 are the same, so we can avoid finding duplicate solutions by lifting that degeneracy
    non_degenerate_n_As = [
        n_As[0],
        n_As[3],
        range(
            sum([n_As[1][0], n_As[2][0], n_As[4][0]]),
            sum([n_As[1][-1], n_As[2][-1], n_As[4][-1]]) + 1,
            n_As[1][0],
        ),
    ]
    non_degenerate_N_As = [N_A[0], N_A[3], N_A[4]]
    non_degenerate_N_Bs = [N_B[0], N_B[3], N_B[4]]
    for _n_As in product(*non_degenerate_n_As):
        n_Bs = [
            m_num * n_A * _N_B // (_N_A * m_denom)
            for n_A, _N_A, _N_B in zip(_n_As, non_degenerate_N_As, non_degenerate_N_Bs)
        ]
        overall_spoilage_rate = Fraction(sum(_n_As) * N_Bs, sum(n_Bs) * N_As)
        if five_per_product_rate == overall_spoilage_rate:
            print(
                "SOLUTION",
                float(overall_spoilage_rate),
                overall_spoilage_rate,
                _n_As,
                n_Bs,
            )
            return


for m, n_As in sorted(five_per_product_ms.items()):
    search_overall_spoilage_rate_solutions(m, n_As)

SOLUTION 1.000677966101695 1476/1475 (2065, 1125, 2750) [252, 738, 3960]
SOLUTION 1.0169491525423728 60/59 (2419, 375, 2706) [300, 250, 3960]
SOLUTION 1.0192090395480227 902/885 (2478, 2025, 3120) [308, 1353, 4576]
SOLUTION 1.0234206471494607 3321/3245 (1947, 5500, 3245) [243, 3690, 4779]
SOLUTION 1.025 41/40 (3640, 3600, 4560) [455, 2419, 6726]
SOLUTION 1.0423728813559323 123/118 (826, 60, 804) [105, 41, 1206]
SOLUTION 1.0677966101694916 63/59 (2419, 3900, 2501) [315, 2730, 3843]
SOLUTION 1.111864406779661 328/295 (59, 900, 65) [8, 656, 104]
SOLUTION 1.1292372881355932 533/472 (1888, 3600, 1272) [260, 2665, 2067]
SOLUTION 1.1371340523882896 738/649 (1298, 4675, 792) [180, 3485, 1296]
SOLUTION 1.1466101694915254 1353/1180 (1652, 1200, 1020) [231, 902, 1683]
SOLUTION 1.1581920903954803 205/177 (354, 1674, 147) [50, 1271, 245]
SOLUTION 1.167457627118644 1722/1475 (885, 1875, 425) [126, 1435, 714]
SOLUTION 1.1813559322033897 697/590 (2124, 4500, 890) [306, 3485, 1513]
SOLUTION 1.191283292